In [ ]:
import sys

# On ajoute le dossier racine dans les chemins de fichiers de python
sys.path.append("../")

In [ ]:
%load_ext autoreload
%autoreload 2

from P9_02_scripts.datasets import *
from P9_02_scripts.models import *
from notebook import *

# Introduction

# Chargement des ressources

## Chargement du workspace

In [ ]:
# On charge l’espace de travail Azure Machine Learning existant
ws = Workspace.from_config()

## Chargement du magasin de données

In [ ]:
# On charge le magasin de données par défaut
datastore = ws.get_default_datastore()

## Chargement de l'environnement d'inférence

In [ ]:
# On spécifie les packages à installer
env_aci = Environment.from_conda_specification(name="env_aci", file_path="conda_aci.yml")

# On enregistre l'environnement
env_aci.register(workspace=ws);

## Chargement des jeux de données

In [ ]:
train_user_article_ratings_ds = Dataset.get_by_name(ws, "train_user_article_ratings")
valid_user_article_ratings_ds = Dataset.get_by_name(ws, "valid_user_article_ratings")
test_user_article_ratings_ds = Dataset.get_by_name(ws, "test_user_article_ratings")

article_profiles_ds = Dataset.get_by_name(ws, "article_profiles")

train_user_profiles_ds = Dataset.get_by_name(ws, "train_user_profiles")

# Développement des modèles

## Modèle baseline

### Entrainement du modèle

In [ ]:
params = {
    # Jeux de données
    "train_user_article_ratings": "train_user_article_ratings",
    "valid_user_article_ratings": "valid_user_article_ratings",
    "article_profiles": "article_profiles",
    
    # Hyperparamètres
    "rating_col": DEFAULT_RATING_COL
}

In [ ]:
from P9_02_scripts.model_baseline_train.run import exp_submit

run = exp_submit(
    ws,
    SCRIPTS_PATH + "model_baseline_train",
    params,
    gs_params=None,
    wait_for_completion=False,
    show_output=False
)

In [ ]:
# # On affiche un widget avec les détails de l'exécution
# RunDetails(run).show()

In [ ]:
# On attend la fin de l'exécution
run.wait_for_completion(show_output=False);

### Analyse des résultats

In [ ]:
# On récupère la dernière exécution de l'expérience.
run = get_last_run(ws, "model_baseline_train")

In [ ]:
# On télécharge le fichier contenant les résultats de l'évaluation du modèle
run.download_file("outputs/res.parquet", PARQUET_PATH + "model_baseline_train_res.parquet")

In [ ]:
# On ouvre et on affiche les résultats
res = pd.read_parquet(PARQUET_PATH + "model_baseline_train_res.parquet")
res

### Enregistrement des hyperparamètres

In [ ]:
with open("../P9_02_scripts/model_baseline_train/params.json", "w") as f:
    json.dump(params, f)

## Modèle de content based filtering

### Cold start problem

### Recherche des hyperparamètres

In [ ]:
params = {
    # Jeux de données
    "train_user_article_ratings": "train_user_article_ratings",
    "valid_user_article_ratings": "valid_user_article_ratings",
    "article_profiles": "article_profiles",
    "train_user_profiles": "train_user_profiles",
    
    # Hyperparamètres
    "rating_col": DEFAULT_RATING_COL
}

gs_params = {
    "num_vars_scale": [0., 0.5, 1.],
    "cat_vars_scale": [0., 0.5, 1.]
}

In [ ]:
from P9_02_scripts.model_content_based_train.run import exp_submit

run = exp_submit(
    ws,
    SCRIPTS_PATH + "model_content_based_train",
    params,
    gs_params=gs_params,
    wait_for_completion=False,
    show_output=False
)

In [ ]:
# # On affiche un widget avec les détails de l'exécution
# RunDetails(run).show()

In [ ]:
# On attend la fin de l'exécution
run.wait_for_completion(show_output=False);

### Analyse des résultats

In [ ]:
# On récupère la dernière exécution de l'expérience.
run = get_last_run(ws, "model_content_based_train")

In [ ]:
# On récupère la meilleure exécution de la recherche des hyperparamètres.
best_run = run.get_best_run_by_primary_metric()

In [ ]:
# On télécharge le fichier contenant les résultats de l'évaluation du modèle
best_run.download_file("outputs/res.parquet", PARQUET_PATH + "model_content_based_train_res.parquet")

In [ ]:
# On ouvre et on affiche les résultats
res = pd.read_parquet(PARQUET_PATH + "model_content_based_train_res.parquet")
res

### Enregistrement des hyperparamètres

In [ ]:
# On récupère les hyperparamètres du meilleur modèle
best_hyperparameters = json.loads(run.get_hyperparameters()[best_run.id])
best_hyperparameters

In [ ]:
# On met en forme les paramètres
best_hyperparameters = {k.replace("--", ""): v for k, v in best_hyperparameters.items()}

In [ ]:
# On met à jour les paramètres de base avec les meilleurs hyperparameters
params.update(best_hyperparameters)

In [ ]:
# On enregistre les paramètres
with open("../P9_02_scripts/model_content_based_train/params.json", "w") as f:
    json.dump(params, f)

## Modèle de collaborative filtering

### Cold start problem

### Recherche des hyperparamètres

In [ ]:
params = {
    # Jeux de données
    "train_user_article_ratings": "train_user_article_ratings",
    "valid_user_article_ratings": "valid_user_article_ratings",
    "article_profiles": "article_profiles"
}

gs_params = {
    "rating_col": ["rating_click_nb", "rating_click_per_session_ratio"],
    "n_factors": [50, 100, 200],
    "n_epochs": [10, 20, 30],
    "lr_all": [0.0001, 0.005, 0.025],
    "reg_all": [0.004, 0.02, 0.1]
}

In [ ]:
from P9_02_scripts.model_collaborative_filtering_train.run import exp_submit

run = exp_submit(
    ws,
    SCRIPTS_PATH + "model_collaborative_filtering_train",
    params,
    gs_params=gs_params,
    wait_for_completion=False,
    show_output=False
)

In [ ]:
# # On affiche un widget avec les détails de l'exécution
# RunDetails(run).show()

In [ ]:
# On attend la fin de l'exécution
run.wait_for_completion(show_output=False);

### Analyse des résultats

In [ ]:
# On récupère la dernière exécution de l'expérience.
run = get_last_run(ws, "model_collaborative_filtering_train")

In [ ]:
# On récupère la meilleure exécution de la recherche des hyperparamètres.
best_run = run.get_best_run_by_primary_metric()

In [ ]:
# On télécharge le fichier contenant les résultats de l'évaluation du modèle
best_run.download_file("outputs/res.parquet", PARQUET_PATH + "model_collaborative_filtering_train_res.parquet")

In [ ]:
# On ouvre et on affiche les résultats
res = pd.read_parquet(PARQUET_PATH + "model_collaborative_filtering_train_res.parquet")
res

### Enregistrement des hyperparamètres

In [ ]:
# On récupère les hyperparamètres du meilleur modèle
best_hyperparameters = json.loads(run.get_hyperparameters()[best_run.id])
best_hyperparameters

In [ ]:
# On met en forme les paramètres
best_hyperparameters = {k.replace("--", ""): v for k, v in best_hyperparameters.items()}

In [ ]:
# On met à jour les paramètres de base avec les meilleurs hyperparameters
params.update(best_hyperparameters)

In [ ]:
# On enregistre les paramètres
with open("../P9_02_scripts/model_collaborative_filtering_train/params.json", "w") as f:
    json.dump(params, f)

# Sélection du meilleur modèle

## Comparaison des résultats

In [ ]:
model_baseline_train_res = pd.read_parquet(PARQUET_PATH + "model_baseline_train_res.parquet")
model_content_based_train_res = pd.read_parquet(PARQUET_PATH + "model_content_based_train_res.parquet")
model_collaborative_filtering_train_res = pd.read_parquet(PARQUET_PATH + "model_collaborative_filtering_train_res.parquet")

In [ ]:
# On réunit tous les résultats
valid_res = pd.concat([
    model_baseline_train_res,
    model_content_based_train_res,
    model_collaborative_filtering_train_res
])

# On classe les modèles en fonction du meilleur recall obtenu sur le jeu de validation
valid_res = valid_res.sort_values("recall@5", ascending=False).reset_index(drop=True)

# On modifie le nom des colonnes
valid_res = valid_res.rename(columns={
    "precision@5": "valid_precision@5",
    "recall@5": "valid_recall@5",
})
valid_res

## Enregistrement du modèle

In [ ]:
# On récupère la dernière exécution de l'expérience.
run = get_last_run(ws, "model_content_based_train")

In [ ]:
# On récupère la meilleure exécution de la recherche des hyperparamètres.
best_run = run.get_best_run_by_primary_metric()

In [ ]:
# On enregistre le modèle
model = best_run.register_model(
    model_name="recommender",
    model_path="outputs/model.joblib",
    tags={"class_name": "ContentBasedRecommender"}
)

# Analyse du modèle

## Chargement du modèle

In [ ]:
# On télécharge les données
model = Model(ws, "recommender")
model_path = model.download(target_dir=MODEL_PATH, exist_ok=True)
model_path

In [ ]:
# import sys
# from scripts import training

# # On inscrit les modules nécessaires au chargement du modèle
# sys.modules['training'] = training

# On charge le modèle
model = joblib.load(model_path)

## Chargement des données de test

In [ ]:
# On charge les datasets dans des DataFrames
test_user_article_ratings = test_user_article_ratings_ds.to_pandas_dataframe()
article_profiles = article_profiles_ds.to_pandas_dataframe()

In [ ]:
# On filtre les colonnes pour obtenir le format : user_id, article_id, rating_id
test_ratings = test_user_article_ratings[["user_id", "article_id", DEFAULT_RATING_COL]]
test_ratings = test_ratings.rename(columns={DEFAULT_RATING_COL: "rating"})

## Evaluation sur le jeu de test

In [ ]:
test_res = get_precision_recall_n_score(
    model,
    "ContentBasedRecommender",
    test_ratings,
    article_profiles,
    top_n=5
)

In [ ]:
# On recomme les colonnes
test_res = test_res.rename(columns={
    "precision@5": "test_precision@5",
    "recall@5": "test_recall@5",
})

# On merge les résultats avec ceux du jeu de validation
pd.merge(valid_res, test_res)

# Déploiement manuel du MVP

<img src="./data/images/MLOps level 0.svg" alt="MLOps level 0.svg" width="900"/>
<p style="text-align: center; text-decoration: underline;">MLOps level 0 (<a href="https://cloud.google.com/architecture/mlops-continuous-delivery-and-automation-pipelines-in-machine-learning">source</a>)</p>

## Déploiement du modèle sur Azure Container Instances

Nous allons utiliser Azure Container Instances. Il s'agit d'une solution simple pour déployer un modèle à titre expérimental. Cette solution est cependant déconseillé par Microsoft pour le déploiement de modèle en production.

Azure va automatiquement créer un servce web et un point de terminaison pour y accéder via une API REST. Quand le service web recevra des tweets à analyser, il les transmettra à un fichier de scoring que l'on doit lui fournir. 

In [ ]:
# On récupère notre modèle
model = Model(ws, "recommender")

In [ ]:
# On crée la configuration pour le déploiement.
aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=1, 
    memory_gb=8,
    description="Recommendation d'articles"
)

# On spécifie l'environnement et le script chargé de prédire
# si un tweet est négatif ou non.
inference_config = InferenceConfig(
    entry_script="recommender_aci.py",
    source_directory=SCRIPTS_PATH,
    environment=env_aci
)

In [ ]:
# On déploie le modèle
model_aci = Model.deploy(
    workspace=ws, 
    name="p9-recommender-aci",
    models=[model], 
    inference_config=inference_config, 
    deployment_config=aciconfig
)

In [ ]:
model_aci.wait_for_deployment(show_output=True)

In [ ]:
print(model_aci.scoring_uri)

## Test du modèle

In [ ]:
data = {
    "user_id": "1234",
    "session_start_dt": datetime(2017, 10, 16, 12).isoformat(),
    "top_n": 5
}

In [ ]:
r = requests.post(model_aci.scoring_uri, json=data)

if not r.ok:
    print(f"Erreur de type {r.status_code}")

In [ ]:
r.json()

## Déploiement du service de prédiction

In [ ]:
dotenv_example_file_path = "P9_03_function/Recommender/.env.example"
dotenv_file_path = "P9_03_function/Recommender/.env"

# On crée le fichier .env dans le dossier de l'azure function
copy2(dotenv_example_file_path, dotenv_file_path)

In [ ]:
# On met à jour dans le fichier .env la variable qui représente
# l'url du service de prédiction déployé sur ACI.
dotenv.set_key(dotenv_file_path, "MODEL_ACI_URL", model_aci.scoring_uri)

Création et déploiement de l'azure function :
- `conda activate p9`
- `./function_create.sh`
- `./function_deploy.sh`

## Test du service de prédiction

In [ ]:
# On lit les configurations
config_parser = ConfigParser()
config_parser.read("function_config.txt")

# On récupère le nom du site
azure_function_name = config_parser.get("DEFAULT", "functionAppName").strip('"')
print(f"Le nom de l'azure function est : {azure_function_name}")

In [ ]:
data = {"userId": "1234"}

In [ ]:
r = requests.post(f"https://{azure_function_name}.azurewebsites.net/api/recommender", json=data)

if not r.ok:
    print(f"Erreur de type {r.status_code}")

In [ ]:
r.json()

## Nettoyage des ressources

On pensera à supprimer le service web à la fin de cette démonstration pour éviter des coût inutiles.

In [ ]:
# Suppression du groupe de ressources du modèle déployé sur ACI
model_aci.delete()

Suppression du groupe de ressources du service de prédiction
- `conda activate p9`
- `./function_delete.sh`

# Architecture de déploiement automatisé

<img src="./data/images/MLOps level 1.svg" alt="MLOps level 1.svg" width="900"/>
<p style="text-align: center; text-decoration: underline;">MLOps level 1 (<a href="https://cloud.google.com/architecture/mlops-continuous-delivery-and-automation-pipelines-in-machine-learning">source</a>)</p>

# Pour aller plus loin